In [ ]:
import ee
import geemap

In [ ]:
Map = geemap.Map(center=(10.48, 37.57), zoom=11)

In [7]:
# Import Geometry
shimburit = ee.FeatureCollection("projects/ee-yilkalgebeyehu/assets/Shimburit_Boundary")

In [31]:
# Set the start date and end date of your time series:
start_date = '2023-04-01'
end_date = '2023-04-30'

# Import CHIRPS daily precipitation
PCP = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")\
        .select("precipitation")\
        .filterDate(start_date, end_date)  # Use both start and end dates

# Calculate the Seasonal precipitation
PCP = PCP.reduce(ee.Reducer.sum())

print(PCP.getInfo())

{'type': 'Image', 'bands': [{'id': 'precipitation_sum', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [32]:
# Reduce the region. The region parameter is the Feature geometry
meanDictionary = PCP.reduceRegion(**{
        "reducer": ee.Reducer.mean(),
        "geometry": shimburit,
        "scale": 5000,
        "maxPixels": 1e9,
    }
)

# The result is a Dictionary.  Print it.
pcp = meanDictionary.get("precipitation_sum")
print("PCP (mm)", pcp.getInfo())

PCP (mm) 84.4602508544922


In [ ]:
# Add the seasonal precipitation to the map
Map.addLayer(PCP, {}, 'Seasonal Precipitation')

In [ ]:
roi = Shimburit.geometry()

In [ ]:
geemap.ee_export_image(PCP, "PCP22.tif", scale=30, region=roi)
